Importing packages

In [ ]:
#Install packages
import pandas as pd
import numpy as np
import os 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA

In [ ]:
riskdata = os.path.join(os.getcwd(), "RiskCodingChallenge.csv")
df = pd.read_csv(riskdata)

df.head()

In [ ]:
df.describe()

Below, the data is getting cleaned.

In [ ]:
#Cleaning the data
columns_to_clean = ['x1', 'x2', 'x3','x4']
df[columns_to_clean] = df[columns_to_clean].apply(pd.to_numeric, errors='coerce')

cleaned_file = 'cleaned_data.csv'
df.to_csv(cleaned_file, index=False)

Gathering the coefficients by combining my linear algebra and machine learning knowledge to successfully train and fit the 5 chosen models.

In [ ]:
#Adds a column of ones for the intercept
X = np.c_[np.ones(df.shape[0]), df[['x1', 'x2', 'x3', 'x4']].values]

#Data column 'y' is the one we want to predict
y = df['y'].values

#Splitting training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Standardize the features (important for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Initializing all models
linear_reg = LinearRegression()
decision_tree_reg = DecisionTreeRegressor()
random_forest_reg = RandomForestRegressor()
gradient_boosting_reg = GradientBoostingRegressor()

#Build the neural network model
mlp_reg = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=42)

#Training all models
linear_reg.fit(X_train,y_train)
decision_tree_reg.fit(X_train,y_train)
random_forest_reg.fit(X_train,y_train)
gradient_boosting_reg.fit(X_train,y_train)
mlp_reg.fit(X_train, y_train)

#Projecting the coefficients using np.dot() and np.linalg.inv()
projection_matrix = np.dot(np.linalg.inv(np.dot(X_train.T, X_train)), X_train.T)
coefficients_projected = np.dot(projection_matrix.T, linear_reg.coef_)

#Extracting only the coefficients for 'x1', 'x2', 'x3', and 'x4'
projected_coefficients = coefficients_projected

print("Original Coefficients:", linear_reg.coef_[0:5])

print("Projected Coefficients:", projected_coefficients[0:4])

#Evaluating all models
y_pred_linear = linear_reg.predict(X_test)
r_squared_linear = r2_score(y_test, y_pred_linear)
print("R-squared for linear:", r_squared_linear)
y_pred_decision_tree = decision_tree_reg.predict(X_test)
r_squared_decision_tree = r2_score(y_test, y_pred_decision_tree)
print("R-squared for decision tree:", r_squared_decision_tree)
y_pred_random_forest = random_forest_reg.predict(X_test)
r_squared_random_forest = r2_score(y_test, y_pred_random_forest)
print("R-squared for random forest:", r_squared_random_forest)
y_pred_gradient_boosting = gradient_boosting_reg.predict(X_test)
r_squared_gradient_boosting = r2_score(y_test, y_pred_gradient_boosting)
print("R-squared for gradient boosting:", r_squared_gradient_boosting)
y_pred_mlp = mlp_reg.predict(X_test)
r_squared_mlp = r2_score(y_test, y_pred_mlp)
print("R-squared for MLP:", r_squared_mlp)

#R-squared, also known as the coefficient of determination, represents the proportion of the dependent variable's variance that
# is captured by the independent variables in the model. A higher R-squared indicates a better fit, 
#implying that a larger percentage of the variability in the response variable is accounted for by the model.

# Print the predicted y values for each model
print("Predicted y values for Linear Regression:", y_pred_linear)
print("Predicted y values for Decision Tree:", y_pred_decision_tree)
print("Predicted y values for Random Forest:", y_pred_random_forest)
print("Predicted y values for Gradient Boosting:", y_pred_gradient_boosting)
print("Predicted y values for MLP:", y_pred_mlp)

#I can now compare the performance of the different models. 
#The model that has the highest R squared will be chosen. 
#The final model will finally be trained and applied to make predictions on new data. 
best_model = max(r_squared_linear, r_squared_decision_tree, r_squared_random_forest, r_squared_gradient_boosting, r_squared_mlp)
print("Best Model (highest R-Squared):", best_model)

Utilizing R squared values, the best model was the MLP model. MLP (Multilayer Perceptron) is a type of artificial neural network and is considered a powerful and versatile model for various machine learning tasks.

In [ ]:
from sklearn.feature_selection import f_regression
#Perform hypothesis test using an F-test
f_statistic, p_value = f_regression(X[:, 1:], y)

#Display results
print("R Squared:", best_model)
print("F-Statistic:", f_statistic)
print("P-Value:", p_value)

#Check if the p-value is less than the significance level (alpha)
alpha = 0.05
if p_value[0] < alpha:
    print("Reject the null hypothesis. The overall model has significant predictive power.")
else:
    print("Fail to reject the null hypothesis. The overall model may not have significant predictive power.")

Being able to reject the null hypothesis proves the "best model" we received early has significant predictive power.

I chose Gaussian Process Regressor as my non-linear model because Gaussian processes are non-parametric models, meaning they can capture complex and non-linear relationships in the data. This flexibility is beneficial when dealing with datasets that exhibit intricate patterns or nonlinear dependencies between features and the target variable.

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, RBF
from scipy.stats import norm, t
from sklearn.model_selection import cross_val_score

# Define the Gaussian Process kernel
kernel = ConstantKernel(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Create the Gaussian Process Regressor
gp_model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)

# Fit the Gaussian Process model
gp_model.fit(X_train, y_train)

# Define negative log-likelihood as a custom scoring function
def neg_log_likelihood(estimator, X, y):
    y_pred, sigma = estimator.predict(X, return_std=True)
    nll = -0.5 * np.sum(np.log(2 * np.pi * sigma**2) + ((y - y_pred) / sigma)**2)
    return nll

# Evaluate the model using negative log-likelihood
nll = cross_val_score(gp_model, X_test, y_test, cv=5, scoring=neg_log_likelihood).mean()
print("Negative Log-Likelihood:", nll)

# Perform confidence interval to show predictive power for the models using confidence level of α = 0.05
alpha = 0.05
z_value = t.ppf(1 - alpha / 2, len(y_test) - 1)
y_pred, sigma = gp_model.predict(X_test, return_std=True)
margin_of_error = z_value * sigma
critical_value = norm.ppf(1 - alpha / 2)

if abs(z_value) > critical_value:
    print(f"The Z-value {z_value} is statistically significant at the {1 - alpha:.0%} confidence level.")
else:
    print(f"The Z-value {z_value} is not statistically significant at the {1 - alpha:.0%} confidence level.")

Conclusion

In conclusion, the provided code demonstrates the application of various regression models, including linear regression, decision tree regression, random forest regression, gradient boosting regression, and a multi-layer perceptron (MLP) neural network, to predict a target variable based on a set of input features. The models are trained and evaluated using R-squared. The selection of the best model is based on the highest R squared, which is a key metric for assessing the accuracy of the regression models.